## Распознавание именованных сущностей (Named Entity Recognition)

([Задание для семинара](#scrollTo=6r75FhEHoBu5&line=2&uniqifier=1), 
[Домашнее задание](#scrollTo=7svbV7-GaLX3&line=7&uniqifier=1))

Именованная сущность, - это объект реального мира, у которого есть имя: человек, организация, географический объект, медицинское обозначение, денежная сумма и т.д. 

NER состоит из двух этапов:
    1. Выделение сущностей в тексте
    2. Назначение каждой сущности типа
    


In [1]:
text = "Губернатор Приморского края Олег Кожемяко рекомендовал администрации Владивостока \
уволить сотрудников, ответственных за установку гранитных лавочек на площади Борцов Революции. \
В Минпросвещения рассказали о разработке российского аналога TikTok для школьников."

#### Stanza 

In [2]:
#! pip install stanza

In [3]:
import stanza
stanza.download('ru')
nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')

2022-12-03 21:45:51 INFO: Downloading default packages for language: ru (Russian) ...


KeyboardInterrupt: 

In [ ]:
doc = nlp(text)
print(text)
doc.sentences[0].ents

#### Polyglot 

In [ ]:
! apt install libicu-dev

In [ ]:
! pip install pyicu pycld2 morfessor polyglot

In [ ]:
! polyglot download embeddings2.ru ner2.ru

In [ ]:
from polyglot.text import Text

Text(text).entities

Detector is not able to detect the language reliably.


[I-LOC(['Приморского', 'края']),
 I-PER(['Приморского', 'края', 'Олег', 'Кожемяко']),
 I-LOC(['Владивостока'])]

In [ ]:
for sent in Text(text).sentences:
  print(sent, "\n")
  for entity in sent.entities:
    print({'tag': entity.tag,
           'entity': entity,
           'start_token': entity.start,
           'end_token': entity.end})

Detector is not able to detect the language reliably.


Губернатор Приморского края Олег Кожемяко рекомендовал администрации Владивостока уволить сотрудников, ответственных за установку гранитных лавочек на площади Борцов Революции. 

{'tag': 'I-LOC', 'entity': I-LOC(['Приморского', 'края']), 'start_token': 1, 'end_token': 3}
{'tag': 'I-PER', 'entity': I-PER(['Приморского', 'края', 'Олег', 'Кожемяко']), 'start_token': 1, 'end_token': 5}
{'tag': 'I-LOC', 'entity': I-LOC(['Владивостока']), 'start_token': 7, 'end_token': 8}
В Минпросвещения рассказали о разработке российского аналога TikTok для школьников. 



In [ ]:
#! pip install natasha

In [3]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

doc = Doc(text)

In [4]:
doc.segment(segmenter)
doc.tag_ner(ner_tagger)

### Применение
**Задание (Семинар, 2 балла):** маскируйте все адреса и имена в электронном письме:

In [5]:
email = """
Уважаемая Эльвира Геннадьевна,

Как шеф-повар, я должен выразить свою обеспокоенность слухами о появлении плесени вблизи некоторых кухонь в корпусе 13 по адресу: Льва Толстого, 42. Обращаю ваше внимание на то, что любые продукты, обрабатываемые, приготавливаемые или потребляемые вблизи плесени, могут быть заражены и небезопасны для употребления.

Если эти сообщения найдут подтверждения, мне придется закрыть несколько столовых. Возможно распределение нагрузки на кухни в корпусах:
№9 площадь Гагарина, 99
№10 улица Южная, 10
№11 Кривой проспект, 17

С уважением,
Алексей Мартынов.
martyn@supercorp.ru
"""


In [6]:
doc1 = Doc(email)
doc1.segment(segmenter)
doc1.tag_ner(ner_tagger)

In [7]:
print(doc1.spans)

[DocSpan(start=11, stop=30, type='PER', text='Эльвира Геннадьевна', tokens=[...]), DocSpan(start=163, stop=176, type='LOC', text='Льва Толстого', tokens=[...]), DocSpan(start=496, stop=504, type='LOC', text='Гагарина', tokens=[...]), DocSpan(start=519, stop=524, type='LOC', text='Южная', tokens=[...]), DocSpan(start=533, stop=548, type='LOC', text='Кривой проспект', tokens=[...]), DocSpan(start=567, stop=583, type='PER', text='Алексей Мартынов', tokens=[...])]


In [9]:
ans = email
l = 0
for s in doc1.spans:
    label = s.type
    if (label == "PER" or label == "LOC"):
        ans = ans[:s.start - l] + label + ans[s.stop - l:]
        l += len(s.text) - 3 #l используется для того, чтобы учитывать длину строк, которые мы вырезали из изначального текста
print(ans)


Уважаемая PER,

Как шеф-повар, я должен выразить свою обеспокоенность слухами о появлении плесени вблизи некоторых кухонь в корпусе 13 по адресу: LOC, 42. Обращаю ваше внимание на то, что любые продукты, обрабатываемые, приготавливаемые или потребляемые вблизи плесени, могут быть заражены и небезопасны для употребления.

Если эти сообщения найдут подтверждения, мне придется закрыть несколько столовых. Возможно распределение нагрузки на кухни в корпусах:
№9 площадь LOC, 99
№10 улица LOC, 10
№11 LOC, 17

С уважением,
PER.
martyn@supercorp.ru



### Обучение теггера с нуля 

Допустим, из текста нужно выделить не имена и места действия, а названия лекарств  (или товаров на онлайн-доске объявлений, или химических соединений).
Если возникла необходимость в выделении именованных сущностей, для которых
не предусмотрены готовые решения (например, данные и множество тегов не похожи на  таковые для новостных текстов на русском языке), при наличии размеченной выборки  можно создать модель NER методами машинного обучения с учителем.

Данные для обучения NER аннотируются с помощью [IOB-тегов](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)) или BILOU-тегов:

    - I = Inside, указывается тип
    - O = Outside
    - B = Begin, указывается тип

В BILOU также есть теги конца сегмента и тег для сегмента из одного слова.
    
Набор типов в предложенном датасете:

    - geo = Geographical Entity
    - org = Organization
    - per = Person
    - gpe = Geopolitical Entity
    - tim = Time indicator
    - art = Artifact
    - eve = Event
    - nat = Natural Phenomenon

In [32]:
! wget https://raw.githubusercontent.com/mrm8488/NER-English/master/ner_dataset.csv

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/peter/.wget-hsts'. HSTS will be disabled.
--2022-12-03 22:19:24--  https://raw.githubusercontent.com/mrm8488/NER-English/master/ner_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15208151 (15M) [text/plain]
Saving to: ‘ner_dataset.csv’

ner_dataset.csv     100%[===================>]  14.50M  2.11MB/s    in 7.9s    

2022-12-03 22:19:34 (1.84 MB/s) - ‘ner_dataset.csv’ saved [15208151/15208151]



In [10]:
import pandas as pd

data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")

In [11]:
data.head(70).tail(16)

,Sentence #,Word,POS,Tag
54,Sentence: 3,They,PRP,O
55,Sentence: 3,marched,VBD,O
56,Sentence: 3,from,IN,O
57,Sentence: 3,the,DT,O
58,Sentence: 3,Houses,NNS,O
59,Sentence: 3,of,IN,O
60,Sentence: 3,Parliament,NN,O
61,Sentence: 3,to,TO,O
62,Sentence: 3,a,DT,O
63,Sentence: 3,rally,NN,O


In [12]:
agg_func = lambda s: [[w, p, t] for w, p, t in zip(s["Word"].values.tolist(),
                                                   s["POS"].values.tolist(),
                                                   s["Tag"].values.tolist())]
grouped = data.groupby("Sentence #").apply(agg_func).values.tolist()

In [13]:
grouped[0]

[['Thousands', 'NNS', 'O'],
 ['of', 'IN', 'O'],
 ['demonstrators', 'NNS', 'O'],
 ['have', 'VBP', 'O'],
 ['marched', 'VBN', 'O'],
 ['through', 'IN', 'O'],
 ['London', 'NNP', 'B-geo'],
 ['to', 'TO', 'O'],
 ['protest', 'VB', 'O'],
 ['the', 'DT', 'O'],
 ['war', 'NN', 'O'],
 ['in', 'IN', 'O'],
 ['Iraq', 'NNP', 'B-geo'],
 ['and', 'CC', 'O'],
 ['demand', 'VB', 'O'],
 ['the', 'DT', 'O'],
 ['withdrawal', 'NN', 'O'],
 ['of', 'IN', 'O'],
 ['British', 'JJ', 'B-gpe'],
 ['troops', 'NNS', 'O'],
 ['from', 'IN', 'O'],
 ['that', 'DT', 'O'],
 ['country', 'NN', 'O'],
 ['.', '.', 'O']]

In [14]:
X_list = [[word[:2] for word in sentence] for sentence in grouped]
y_list = [[word[2] for word in sentence] for sentence in grouped]

In [15]:
print(X_list[0], '\n')
print(y_list[0])

[['Thousands', 'NNS'], ['of', 'IN'], ['demonstrators', 'NNS'], ['have', 'VBP'], ['marched', 'VBN'], ['through', 'IN'], ['London', 'NNP'], ['to', 'TO'], ['protest', 'VB'], ['the', 'DT'], ['war', 'NN'], ['in', 'IN'], ['Iraq', 'NNP'], ['and', 'CC'], ['demand', 'VB'], ['the', 'DT'], ['withdrawal', 'NN'], ['of', 'IN'], ['British', 'JJ'], ['troops', 'NNS'], ['from', 'IN'], ['that', 'DT'], ['country', 'NN'], ['.', '.']] 

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [16]:
from sklearn.model_selection import train_test_split

data_train, data_test, y_train, y_test = train_test_split(X_list, y_list, test_size=0.2, random_state=1337)

In [81]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        #'bias': 0.1,
        'word.lower()': word.lower(),
        #'word[-3:]': word[-3:],
        #'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        #'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            #'-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            #'+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [82]:
X_train = [sent2features(s) for s in data_train]
X_test = [sent2features(s) for s in data_test]

## Домашнее задание (6 баллов)

Обучите модель для распознавания именованных сущностей с помощью метода условных случайных полей на классических признаках. Используйте библиотеку ``sklearn_crfsuite``. Экспериментируйте с признаками. Приведите результаты анализа признаков обученной модели.

Решения в форме ноутбуков, где последняя ячейка содержит ответ с текстовым комментарием, присылайте на mipttextanalysis20@gmail.com (ссылкой на colab.research или файлом). 

Решения без штрафа (8 баллов максимум) принимаются до 12.00 6 октября. Всё, что присылается не в срок, оценивается из максимума 4 балла. Работу, присланную в срок и оцененную не менее чем на 4 балла, можно доделать и досдать на максимум один раз.

Пожалуйста, указывайте фамилию в названии блокнота. Комментарий к решению принимается на русском или английском языке.

In [83]:
#! pip install --upgrade 'scikit-learn==0.23.2'

In [84]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [85]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-org',
 'I-org',
 'B-gpe',
 'B-tim',
 'I-tim',
 'B-per',
 'I-per',
 'B-geo',
 'I-geo',
 'B-eve',
 'B-nat',
 'I-eve',
 'I-nat',
 'B-art',
 'I-art',
 'I-gpe']

In [86]:
from sklearn_crfsuite import metrics
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.8579102406446253

In [87]:
from sklearn_crfsuite import metrics
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-art      0.462     0.097     0.160        62
       I-art      0.500     0.103     0.171        58
       B-eve      0.694     0.521     0.595        48
       I-eve      0.467     0.259     0.333        27
       B-geo      0.861     0.911     0.885      7591
       I-geo      0.827     0.787     0.807      1572
       B-gpe      0.972     0.934     0.952      3211
       I-gpe      0.963     0.578     0.722        45
       B-nat      0.550     0.314     0.400        35
       I-nat      0.500     0.333     0.400         6
       B-org      0.803     0.753     0.777      4018
       I-org      0.822     0.813     0.818      3372
       B-per      0.858     0.831     0.844      3362
       I-per      0.860     0.908     0.883      3432
       B-tim      0.937     0.879     0.907      4086
       I-tim      0.861     0.747     0.800      1354

   micro avg      0.867     0.852     0.860     32279
   macro avg      0.746   

In [88]:
import eli5
eli5.show_weights(crf, top=10)

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,6.173,0.734,-3.723,1.632,-3.06,2.58,-6.477,0.858,-2.764,0.797,-2.305,2.492,-4.961,2.928,-4.716,2.965,-5.223
B-art,0.483,-0.216,5.489,0.0,-0.329,-0.724,-1.278,-1.639,-0.65,0.0,0.0,0.124,-1.323,-1.691,-2.04,-0.416,-1.2
I-art,0.224,-0.897,5.216,0.0,-0.031,-0.486,-1.386,-1.38,-0.378,0.0,0.0,-1.306,-0.983,-2.311,-2.143,-1.39,-0.877
B-eve,-0.927,-0.091,-0.649,-0.6,5.674,-1.375,-1.323,-2.05,-0.793,-0.529,-0.131,-1.851,-1.252,-2.658,-2.037,-0.946,-1.663
I-eve,0.181,-0.06,-0.516,-2.788,5.182,-1.354,-1.265,-1.291,-0.408,0.0,0.0,-1.646,-0.921,-2.397,-1.787,-1.58,-1.591
B-geo,0.848,0.0,-3.045,-0.361,-1.996,-4.427,4.561,-0.61,-4.197,-0.998,-1.105,-0.813,-3.836,-2.076,-4.812,0.98,-3.51
I-geo,0.361,0.433,-2.079,-1.068,-1.185,-3.226,3.513,-1.718,-2.497,-0.033,-0.541,-1.301,-2.78,-1.609,-3.673,0.239,-2.606
B-gpe,0.93,-3.351,-3.672,-1.54,-2.905,-0.779,-5.059,-7.155,3.232,-1.379,-1.797,0.253,-4.575,-1.557,-5.149,-1.173,-3.333
I-gpe,-0.033,-0.558,-0.293,0.0,0.0,-0.917,-1.295,-1.828,3.474,0.0,0.0,-2.49,-0.951,-1.456,-1.137,-1.792,-1.208
B-nat,-0.014,0.0,-0.111,0.0,0.0,0.19,-0.372,-0.788,-0.008,-0.651,6.425,-0.824,-0.19,-1.432,-1.575,-0.541,-0.431


### Комментарий

Наилучшая результативность у модели достигается при использовании word.isdigit, word.postag, word.lower, word.isupper, word.istitle;
при использовании информации о соседних словах точность понижается;
также в таблице сверху представлены веса переходов между тэгами, а также топ признаков по тэгам